In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/reto_life_clef/Modelo

ColabNotebooks
PlantCLEF
PlantCLEF2025_pseudoquadrats_without_labels_complementary_training_set
PlantCLEF2025_test_images
pretrained_models


In [ ]:
import torch

model_path = '/content/drive/MyDrive/reto_life_clef/Modelo/pretrained_models/vit_base_patch14_reg4_dinov2_lvd142m_pc24_onlyclassifier_then_all/model_best.pth.tar'

try:
  model = torch.load(model_path, weights_only=False)
  print("Modelo cargado exitosamente.")

except FileNotFoundError:
  print(f"Error: Archivo no encontrado en {model_path}. Verifica la ruta.")
except Exception as e:
  print(f"Error al cargar el modelo: {e}")

Modelo cargado exitosamente.


In [ ]:
## Average only over the patches in which the species occurs

In [ ]:
from argparse import ArgumentParser
import pandas as pd
from urllib.request import urlopen
from PIL import Image
import timm
import torch

In [ ]:
class Args:
    image = "/content/drive/MyDrive/reto_life_clef/Modelo/PlantCLEF2025_test_images"
    class_mapping = "/content/drive/MyDrive/reto_life_clef/Modelo/pretrained_models/class_mapping.txt"
    species_mapping = "/content/drive/MyDrive/reto_life_clef/Modelo/pretrained_models/species_id_to_name.txt"
    pretrained_path = "/content/drive/MyDrive/reto_life_clef/Modelo/pretrained_models/vit_base_patch14_reg4_dinov2_lvd142m_pc24_onlyclassifier_then_all/model_best.pth.tar"
    device = "cuda"

args = Args()

In [ ]:
def load_class_mapping(class_list_file):
    """Carga el mapping de índice de clase a ID de especie."""
    with open(class_list_file) as f:
        class_index_to_class_name = {i: line.strip() for i, line in enumerate(f)}
    return class_index_to_class_name

In [ ]:
def load_species_mapping(species_map_file):
    """Carga el mapping de ID de especie a nombre de especie."""
    df = pd.read_csv(species_map_file, sep=';', quoting=1, dtype={'species_id': str})
    df = df.set_index('species_id')
    return df['species'].to_dict()

In [ ]:
from PIL import Image

def split_image(image_path, filas= 4, cols= 2, output_prefix="parte"):
    """
      Divide the image into (rows x cols) equal parts and save
      them as <output_prefix>_1.jpg … <output_prefix>_{rows*cols}.jpg
    """
    imagen = Image.open(image_path)
    ancho, alto = imagen.size

    # Use the rows and cols parameters
    ancho_parte = ancho // cols
    alto_parte  = alto  // filas

    contador = 1
    for i in range(filas):
        for j in range(cols):
            izquierda = j * ancho_parte
            superior  = i * alto_parte
            derecha   = izquierda + ancho_parte
            inferior  = superior  + alto_parte

            parte = imagen.crop((izquierda, superior, derecha, inferior))
            parte.save(f"{output_prefix}_{contador}.jpg")
            contador += 1

In [ ]:
# Loading mappings and model
cid_to_spid = load_class_mapping(args.class_mapping)
spid_to_sp  = load_species_mapping(args.species_mapping)
device      = torch.device(args.device)

model = timm.create_model(
    'vit_base_patch14_reg4_dinov2.lvd142m',
    pretrained=False,
    num_classes=len(cid_to_spid),
    checkpoint_path=args.pretrained_path
    ).to(device).eval()

data_config = timm.data.resolve_model_data_config(model)
transforms  = timm.data.create_transform(**data_config, is_training=False)

In [ ]:
import os
import glob
import csv
from tqdm import tqdm

# Image directory
test_dir = "/content/drive/MyDrive/reto_life_clef/Modelo/PlantCLEF2025_test_images"
image_paths = sorted(glob.glob(os.path.join(test_dir, "*.jpg")))

submission = []
# Image-level progress bar
for image_path in tqdm(image_paths, desc="Procesando imágenes", unit="imagen"):
    quadrat_id = os.path.splitext(os.path.basename(image_path))[0]
    # Initializes accumulators
    agg_probs, agg_counts = {}, {}

    # Divide the image into patches
    imagen = Image.open(image_path)
    filas, cols = 4, 2
    ancho, alto = imagen.size
    pw, ph = ancho // cols, alto // filas

    for i in range(filas):
        for j in range(cols):
            patch = imagen.crop((j*pw, i*ph, (j+1)*pw, (i+1)*ph))
            tensor = transforms(patch).unsqueeze(0).to(device)
            out = model(tensor)
            top_p, top_i = torch.topk(out.softmax(dim=1) * 100, k=10)
            probs = top_p.cpu().detach().numpy()[0]
            idxs = top_i.cpu().detach().numpy()[0]
            # Accumulate probabilities and counts by species
            for p, cid in zip(probs, idxs):
                spid = cid_to_spid[cid]
                agg_probs[spid] = agg_probs.get(spid, 0.0) + float(p)
                agg_counts[spid] = agg_counts.get(spid, 0) + 1

    # Calculate average and filter > 5%
    avg = {spid: agg_probs[spid]/agg_counts[spid]
           for spid in agg_counts
           if (agg_probs[spid]/agg_counts[spid]) > 5}

    # Sort species by trust
    species_ids = sorted(avg, key=avg.get, reverse=True)
    species_str = f"[[{', '.join(species_ids)}]]" if species_ids else "[[]]"
    submission.append([quadrat_id, species_str])

# Save the result in CSV with quotes and without index
df_sub = pd.DataFrame(submission, columns=["quadrat_id", "species_ids"])
df_sub.to_csv("predicciones.csv", index=False, quoting=csv.QUOTE_ALL)

print("Predicciones listas")

Procesando imágenes: 100%|██████████| 2105/2105 [52:51<00:00,  1.51s/imagen]

Predicciones listas
